# Wave Equation

We consider the previous geometry.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *   # Opencascade for geometry modeling

In [2]:
outer = Rectangle(2, 2).Face()
outer.edges.name="outer"
outer.edges.Max(X).name = "right"
outer.edges.Min(X).name = "left"
outer.edges.Min(Y).name = "bottom"
outer.edges.Max(Y).name = "top"

# We create a smaller rectangle inside the large one. 

inner = MoveTo(1, 0.9).Rectangle(0.3, 0.5).Face()
inner.edges.name="interface"
inner.maxh = 0.05

# We also create a circle inside the large one
circle = MoveTo(0.5, 0.5).Circle(0.25).Face()
circle.edges.name="circle-interface"
circle.maxh = 0.02

# subtraction!

outer = outer - inner - circle 

inner.faces.name="inner"
inner.faces.col = (1, 0, 0) # color
circle.faces.name = "circle"
circle.faces.col = (0,0,1)
outer.faces.name="outer"

geo = Glue([outer])
Draw(geo);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [9]:
# This maxh is a global one.

mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.05)) 
Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

## Complex Valued FES

In [10]:
fes = H1(mesh, order=5, complex=True) # Now we need complex values
u, v = fes.TnT()

## Gaussian Pulse Function
The function $f$ is a Gaussian pulse.

In [17]:
k = 50 # frequency
pulse = 400*exp(-(20**2)*((x-0.8)*(x-0.8) + (y-1)*(y-1)))
Draw(pulse, mesh, order=5)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

## Weak Form

In [18]:
# Forms
a = BilinearForm(fes)
a += grad(u)*grad(v)*dx - k**2*u*v*dx
a += -k*1j*u*v * ds("outer") # boundary integral!
a.Assemble()

f = LinearForm(pulse * v * dx).Assemble();

## Solve

In [19]:
sol = GridFunction(fes)
sol.vec.data = a.mat.Inverse() * f.vec
Draw(sol, mesh, order=5);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…